## Investigate wflow outputs with xarray

In [ ]:
%matplotlib inline
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
fn = '/home/hcwinsemius/Barotse/zambezi_4km/all_outputs/outmaps.nc'
fn = '/home/hcwinsemius/Barotse/zambezi_4km/all_outputs_Minf_Ksat10_SWC4_RootDepth_4/outmaps.nc'
ds = xr.open_dataset(fn)
ds

In [ ]:
# plot some spatial sums
import numpy as np
f = plt.figure(figsize=(16,16))
ssf = ds['ssf_riv'].sum(dim='time')
qo = ds['qo_riv'].sum(dim='time')
# Inwater = dss[-1]['Inwater'].sum(dim='time')
qo = qo.where(qo+ssf>0)
ssf = ssf.where(qo+ssf>0)
ssf *= ssf>0
qo *= qo>0
# watbal *=watbal<1750
# dss[-1]['wat'][25].plot()
ax1 = plt.subplot(221)
qo.plot(ax=ax1, cmap='viridis_r')

ax2 = plt.subplot(222)
ssf.plot(ax=ax2, cmap='viridis_r')

ax3 = plt.subplot(223)
(qo+ssf).plot(ax=ax3, cmap='viridis_r')

ax4 = plt.subplot(224)
ds['ex_ss'][53].plot(ax=ax4, cmap='Blues_r')
# wb = watbal.mean(dim=('lat', 'lon'))
# wb.plot()

In [ ]:
import pcraster as pcr
import numpy as np
ldd = pcr.readmap('/home/hcwinsemius/Barotse/zambezi_4km/staticmaps/wflow_ldd.map')
iw = ds['inw'].values

swc = ds['SWC'].values
swd = ds['SWD'].values
ustore = ds['ustore'].values

# iw.shape
accu = [pcr.pcr2numpy(pcr.accuflux(ldd, pcr.numpy2pcr(pcr.Scalar, i, np.nan)), np.nan) for i in iw]

ustorecap = [c-d-u for c, d, u in zip(swc, swd, ustore)]

# add to Dataset
ds['accu'] = (('time', 'lat', 'lon'), accu)
ds['ustorecap'] = (('time', 'lat', 'lon'), ustorecap)

# f = plt.figure(figsize=(16,16))



In [ ]:
import hvplot.xarray
# dss[-1]['zi'].hvplot.quadmesh('lon', 'lat', cmap='Blues_r', rasterize=True, dynamic=True, frame_width=550, frame_height=500, datashade=True, widget_type='scrubber', widget_location='bottom')
# dss[-1]['zi'].hvplot.quadmesh('lon', 'lat', clim=(0, 7000), dynamic=True, frame_width=550, frame_height=500, cmap='Blues_r', widget_type='scrubber', widget_location='bottom', rasterize=True)
# (dss[-1]['SWC']-dss[-1]['SWD']).hvplot.quadmesh('lon', 'lat', dynamic=True, frame_width=550, frame_height=500, cmap='Blues_r', widget_type='scrubber', widget_location='bottom', rasterize=True)
ds['ex_ss'].hvplot.quadmesh('lon', 'lat', dynamic=True, frame_width=550, frame_height=500, cmap='Blues_r', widget_type='scrubber', widget_location='bottom', rasterize=True)


In [ ]:
import hvplot.xarray
# dss[-1]['zi'].hvplot.quadmesh('lon', 'lat', cmap='Blues_r', rasterize=True, dynamic=True, frame_width=550, frame_height=500, widget_type='scrubber', widget_location='bottom')
ds['zi'].hvplot.quadmesh('lon', 'lat', clim=(0, 1500), dynamic=True, frame_width=550, frame_height=500, cmap='Blues_r', widget_type='scrubber', widget_location='bottom', rasterize=True)
# dss[-1]['ustore'].hvplot.quadmesh('lon', 'lat', dynamic=True, frame_width=550, frame_height=500, cmap='Blues_r', widget_type='scrubber', widget_location='bottom', rasterize=True)


In [ ]:
import hvplot.xarray

dss[-1]['qo_riv'].hvplot.quadmesh('lon', 'lat', dynamic=True, frame_width=550, frame_height=500, cmap='Blues_r', widget_type='scrubber', widget_location='bottom', rasterize=True)


In [ ]:
f = plt.figure(figsize=(16, 16))
ax1 = plt.subplot(221)
ax2 = plt.subplot(222)
ax3 = plt.subplot(223)
ax4 = plt.subplot(224)
ax1.grid()
ax2.grid()
ax3.grid()
ax4.grid()
surf_area = 5e5 * 1e6 # (about 500,000 km2)

conv_fact = 86400 * surf_area**-1 * 1e3

zi = -ds['zi'].mean(dim=('lat', 'lon'))
transfer = ds['trans'].mean(dim=('lat', 'lon'))
ssf = ds['ssf_riv'].sum(dim=('lat', 'lon'))
qo = ds['qo_riv'].sum(dim=('lat', 'lon'))
inw = ds['inw'].sum(dim=('lat', 'lon'))
surf_runoff = inw-ssf-qo
tot = ssf + qo

# make plots
(zi).plot(ax=ax1, label='Groundwater level')
ax1.set_ylabel('depth under surface [mm]')

transfer.cumsum().plot(ax=ax2, label='Accumulated transfer from unsaturated to saturated zone')
ax2.set_ylabel('Transfer [mm]')
ax2.legend()

qo.plot(ax=ax3, label='lateral excess outflow from soil')
ssf.plot(ax=ax3, label='sub-surface lateral flow')
surf_runoff.plot(ax=ax3, label='runoff from surface')
ax3.set_ylabel('Lateral flux [m3/s]')
ax3.legend()

inw.plot(ax=ax4, label='Total lateral flow per timestep')
ax4.set_ylabel('Lateral flux [m3/s]')
ax4.legend()

In [ ]:
# in mm
f = plt.figure(figsize=(16, 10))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
# ax3 = plt.subplot(223)
# ax4 = plt.subplot(224)
ax1.grid()
ax2.grid()
# ax3.grid()
# ax4.grid()
surf_area = 3580 * 3688 # 5e5 * 1e6 # (about 500,000 km2)
conv_fact = 86400 * surf_area**-1 * 1e3 # from m3/s to mm day-1
# ds['swf'][100].plot()
zi = -ds['zi'].mean(dim=('lat', 'lon'))
ssf = ds['ssf_riv'].mean(dim=('lat', 'lon'))*conv_fact  # from m3 s-1 to mm per day
qo = ds['qo_riv'].mean(dim=('lat', 'lon'))*conv_fact  # from m3 s-1 to mm per day
ex_us = ds['ex_us'].mean(dim=('lat', 'lon'))
ex_ss = ds['ex_ss'].mean(dim=('lat', 'lon'))
inw = ds['inw'].mean(dim=('lat', 'lon'))*conv_fact
P = ds['P'].mean(dim=('lat', 'lon'))
AET = ds['AET'].mean(dim=('lat', 'lon'))
# tot = ssf + qo
(zi).plot(ax=ax1)
plt.ylabel('depth under surface [mm]')
ssf.cumsum().plot(ax=ax2, label='subsurface flow')
qo.cumsum().plot(ax=ax2, label='land runoff')
ex_us.cumsum().plot(ax=ax2, label='exfilt_us')
ex_ss.cumsum().plot(ax=ax2, label='exfilt_ss')
inw.cumsum().plot(ax=ax2, label='total runoff')
P.cumsum().plot(ax=ax2, color='k', linestyle='-', label='Precip')
AET.cumsum().plot(ax=ax2, color='k', linestyle='--', label='Evaporation')
plt.ylabel('cumulative flux [mm]')
plt.legend()


In [ ]:
# lon = 23.2  # Lukulu
# lat = -14.337 # Lukulu
lon = 24.15 # Watopa
lat = -13.584 # Watopa

ac = ds['accu'].sel(lat=lat, lon=lon, method='nearest')
ts = ds['run'].sel(lat=lat, lon=lon, method='nearest')
ac.plot(label='instantaneous accumulated flow')
ts.plot(label='Routed discharge')
plt.ylabel('Discharge [m3/s]')
plt.legend()
plt.grid()


In [ ]:
ts_month = ts.resample(time='MS').mean()
ac_month = ac.resample(time='MS').mean()

In [ ]:
ac_month.plot(label='Instantaneous accumulated flow')
ts_month.plot(label='Routed discharge')
plt.ylabel('Discharge [m3/s]')
plt.legend()
plt.grid()
